In [4]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [6]:
trainset = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
testset = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

0it [00:00, ?it/s]

100%|███████████████████████████████████████████████████████████████▉| 170483712/170498071 [22:12<00:00, 186320.59it/s]

Extracting data\cifar-10-python.tar.gz to data
Files already downloaded and verified


170500096it [22:30, 186320.59it/s]                                                                                     

In [7]:
trainset = torch.utils.data.DataLoader(
    trainset,
    batch_size=20,
    shuffle=True
)

testset = torch.utils.data.DataLoader(
    testset,
    batch_size=20,
    shuffle=True
)

In [8]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# create a complete CNN
model = Net()
print(model)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [9]:
import torch.optim as optim

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
n_epochs = 8

valid_loss_min = np.Inf

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    valid_loss = 0.0
    
    for data, target in trainset:
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################

    for data, target in testset:
        
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(trainset.dataset)
    valid_loss = valid_loss/len(testset.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.033994 	Validation Loss: 0.954985
Validation loss decreased (inf --> 0.954985).  Saving model ...
Epoch: 2 	Training Loss: 0.882064 	Validation Loss: 0.933560
Validation loss decreased (0.954985 --> 0.933560).  Saving model ...
Epoch: 3 	Training Loss: 0.792492 	Validation Loss: 0.880695
Validation loss decreased (0.933560 --> 0.880695).  Saving model ...
Epoch: 4 	Training Loss: 0.724937 	Validation Loss: 0.836594
Validation loss decreased (0.880695 --> 0.836594).  Saving model ...
Epoch: 5 	Training Loss: 0.668717 	Validation Loss: 0.902325
Epoch: 6 	Training Loss: 0.624337 	Validation Loss: 0.863170
Epoch: 7 	Training Loss: 0.576066 	Validation Loss: 0.904716
Epoch: 8 	Training Loss: 0.550598 	Validation Loss: 0.888815


In [15]:
correct = 0
total = 0

with torch.no_grad():
    for data in trainset:
        X, y = data
        output = model(X)
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
            
print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.829


In [17]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = model(X)
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
            
print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.712
